In [1]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html
from copy import deepcopy

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

········


### Setup Model

In [3]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

### Operating System Environment

In [4]:
environment_prompt_template = '''
Please take on the role of a linux(ubuntu) environment. I will send bash code and commands and you will respond with the output that is as close to the output of the linux environment as possible. You should use the "Environment Information" provided when making your response. Limit the response to around 250 characters and note that the output has been truncated with the message  "[truncated because the output is too long]".  Do not repeat the command provided. 

Environment information:
{}

First command:
{}
'''

agent_prompt_template = '''
Interact with a software environment to solve a task. Imagine you are an intelligent agent working for a user and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given a detailed description of the current environment and your goal to accomplish. For each of your turns, you will be given a list of actions which you can choose one to perform in this turn. You should provide two parts of your response: "THOUGHT" and "ACTION". For  "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your "ACTION" in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.
 ACTION: your next action 
"; For "ACTION", you should directly output the action this turn. Your output must strictly follow this format:"ACTION: your next action
". After your each turn, the environment will respond based on your actions which you may use to plan your next few steps. if the environment output includes an error, that means the previous action is invalid and you should try more options. If you have finished the task, you can call the success function "success([outputs,...])" with any final outputs.
 Reminder: 
1. the action must follow any formats requested
2. Think when necessary, try to act directly more in the process.
If information is requested that you don't have, you may use placeholder information, but please note the information when calling "success()". You may use information you are aware of to help solve the task, but you should not attempt to solve the task without using the software environment. 

Software Environment: {}
Your Task: {}
'''

In [79]:
environment_information_EV_prompt = "The environment is a linux(ubuntu) installation."
software_environment = "You are interacting with a linux(ubuntu) environment. You will use bash code and commands to accomplish your task."
agent_task = "tell me how many files are in the directory '/etc'?"

In [6]:
agent_prompt = agent_prompt_template.format(software_environment, agent_task)


agent_messages = [
    HumanMessage(content=agent_prompt),
]


In [7]:
agent_response = model.predict_messages(agent_messages)

In [8]:
agent_response.content

"THOUGHT: To determine the number of files in '/etc' directory, I can use the 'ls' command to list all files and the 'wc -l' command to count the number of lines. This combination should give me the number of files in the directory.\n\nACTION: ls /etc | wc -l"

In [9]:
agent_result = agent_response.content.split("ACTION:")[1].strip()

In [10]:
agent_result

'ls /etc | wc -l'

In [11]:
environment_prompt = environment_prompt_template.format(environment_information_EV_prompt, agent_result)

environment_messages = [
    HumanMessage(content=environment_prompt)
]

In [12]:
environment_result = model.predict_messages(environment_messages)

In [13]:
environment_result.content

'The output of the command is:\n\n140'

In [14]:
agent_messages.append(HumanMessage(content=environment_result.content))

In [15]:
num_turns = 10

for i in range(num_turns):
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_result = agent_response.content.split("ACTION:")[1].strip()
    environment_messages.append(HumanMessage(content=agent_result))
    agent_messages.append(agent_response)
    if "success(" in agent_result:
        break
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    print(environment_result.content)
    agent_messages.append(HumanMessage(content=environment_result.content))

THOUGHT: The environment has given me a direct answer for how many files are in the directory '/etc'. It's 140. Now, I should call the success function to complete this task.

ACTION: success([140])


#### Get tasks from AgentInstruct data

In [18]:
import pandas as pd
df5 = pd.read_parquet('../../agentinstruct/os-00000-of-00001-971539c34fcc7500.parquet')
print(df5)

                                         conversations      id
0    [{'from': 'human', 'loss': None, 'value': 'You...    os_0
1    [{'from': 'human', 'loss': None, 'value': 'You...    os_1
2    [{'from': 'human', 'loss': None, 'value': 'You...    os_2
3    [{'from': 'human', 'loss': None, 'value': 'You...    os_3
4    [{'from': 'human', 'loss': None, 'value': 'You...    os_4
..                                                 ...     ...
190  [{'from': 'human', 'loss': None, 'value': 'You...  os_190
191  [{'from': 'human', 'loss': None, 'value': 'You...  os_191
192  [{'from': 'human', 'loss': None, 'value': 'You...  os_192
193  [{'from': 'human', 'loss': None, 'value': 'You...  os_193
194  [{'from': 'human', 'loss': None, 'value': 'You...  os_194

[195 rows x 2 columns]


In [28]:
print(df5.iloc[2]["conversations"][6]['value'])

Now, I will start a new problem in a new OS. My problem is:

 

Count how many files or directories in the current directory have the read permission for the user.


In [31]:
df5.iloc[2]["conversations"][6]['value'].split("is:")[1].strip()

'Count how many files or directories in the current directory have the read permission for the user.'

In [32]:
len(df5)

195

In [71]:
def process(data):
    return data['conversations'][6]['value'].split("is:")[1].strip()

In [77]:
tasks = df5.apply(process, axis=1).tolist()

In [78]:
tasks

['Filter out all the empty lines and comments in configurations (.conf) files in the current directory, and find the most frequently appearing valid configuration directive.',
 'Given the contents of a log file named \'system.log\' in the current directory, figure out the total number of occurrences of the word "error".',
 'Count how many files or directories in the current directory have the read permission for the user.',
 'Count the total sum of numbers that lie within the last 100 lines of the text file "numbers.txt", where each line of the file has a single whole number. Report the sum as your answer.',
 'List the usernames of all users on the system who are using bin/bash as their default shell.',
 'In the current directory containing various types of files, your task is to get the combined size of all files ending with `.txt` within the directory and its subdirectories. The file size should be returned in bytes.',
 'Locate in your home directory all the text files (.txt) that co

#### Multiple conversations

In [80]:
num_turns = 20

all_agents = []
all_environments = []

for x in range(len(tasks[:2])): 
    agent_prompt = agent_prompt_template.format(software_environment, agent_task)
    agent_messages = [
        HumanMessage(content=agent_prompt),
    ]
    agent_response = model.predict_messages(agent_messages)
    environment_prompt = environment_prompt_template.format(environment_information_EV_prompt, agent_result)
    environment_messages = [
        HumanMessage(content=environment_prompt)
    ]
    agent_messages.append(HumanMessage(content=environment_result.content))
    for i in range(num_turns):
        agent_response = model.predict_messages(agent_messages)
        print(agent_response.content)
        agent_result = agent_response.content.split("ACTION:")[1].strip()
        environment_messages.append(HumanMessage(content=agent_result))
        agent_messages.append(agent_response)
        if "success(" in agent_result:
            break
        environment_result = model.predict_messages(environment_messages)
        environment_messages.append(environment_result)
        print(environment_result.content)
        agent_messages.append(HumanMessage(content=environment_result.content))
    all_agents.append(deepcopy(agent_messages))
    all_environments.append(deepcopy(environment_messages))

THOUGHT: The user wants to know how many files are in the '/etc' directory. The standard Linux utility for listing files is 'ls'. However, 'ls' output includes both files and directories. To count only files, I'll need to use 'find' command. The '-type f' option can be used with 'find' command to limit the search to files only. Then I can pipe this output to 'wc' command with '-l' option to count the number of lines. 

ACTION: find /etc -type f | wc -l
The command you provided isn't valid. However, I'm assuming you want to know the number of files in the /etc directory. The output would be:

2953
THOUGHT: The environment indicated that my previous command was invalid, but it successfully returned the number of files in the /etc directory by using a different method. 

ACTION: success("There are 2953 files in the /etc directory.")
THOUGHT: To find out the number of files in the /etc directory, I will use the 'ls' command in the Linux shell. This command will list all the files and folde